# Proyecto Individual 1 (MLOPs): Sistema de Recomendación de Películas

## ETL

In [16]:
'''Importamos todas las librerías que se estarán utilizando'''

import pandas as pd
import numpy as np
import calendar
from googletrans import Translator
import datetime as dt
import json

In [17]:
''' Importamos el dataset de movies'''

movies_df = pd.read_csv('Dataset/movies_dataset.csv')
movies_df.head()

C:\Users\Luis\AppData\Local\Temp\ipykernel_8632\3558972948.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('Dataset/movies_dataset.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [18]:
movies_df.shape

(45466, 24)

In [19]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [20]:
''' Importamos el dataset de credits'''

credits_df = pd.read_csv('Dataset/credits.csv')
credits_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [21]:
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [22]:
'''Observamos como esta formada la columna belongs_to_collection'''

movies_df.loc[0, 'belongs_to_collection']

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [23]:
'''Observamos como esta formada la columna genres'''
movies_df.loc[0, 'genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [24]:
movies_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [25]:
'''Se identifican las filas no numéricas de la columna budget:'''

c = []
for i in range(0, len(movies_df)):
    if movies_df.loc[i, 'budget'].isalnum():
        c = c
    else:
        c.append(i)
print(c)

[19730, 29503, 35587]


In [26]:
'''Observamos las 3 filas que no nos permiten realizar el cambio del tipo de dato'''

movies_df.loc[[19730, 29503, 35587]]


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
'''Creamos una función que nos ayudará a realizar las conversiones necesarias para arreglar las filas mencionadas.
    Los cambios que se realizan siguen una secuencia para evitar la pérdida de información. Se debe pasar
    el número de la fila a la función'''

def repair(x):
    movies_df.loc[x, 'status'] = movies_df.loc[x, 'overview']
    movies_df.loc[x, 'title'] = movies_df.loc[x, 'poster_path']
    movies_df.loc[x, 'vote_average'] = movies_df.loc[x, 'production_countries']
    movies_df.loc[x, 'vote_count'] = movies_df.loc[x, 'release_date']
    movies_df.loc[x, 'runtime'] = movies_df.loc[x, 'original_language']
    movies_df.loc[x, 'spoken_languages'] = movies_df.loc[x, 'original_title']
    movies_df.loc[x, 'original_language'] = np.nan
    movies_df.loc[x, 'overview'] = movies_df.loc[x, 'adult']
    movies_df.loc[x, 'poster_path'] = movies_df.loc[x, 'budget']
    movies_df.loc[x, 'production_countries'] = movies_df.loc[x, 'homepage']
    movies_df.loc[x, 'release_date'] = movies_df.loc[x, 'id']
    movies_df.loc[x, 'original_title'] = movies_df.loc[x, 'popularity']
    movies_df.loc[x, 'id'] = np.nan
    movies_df.loc[x, 'homepage'] = np.nan
    movies_df.loc[x, 'adult'] = movies_df.loc[x, 'production_companies']
    movies_df.loc[x, 'budget'] = movies_df.loc[x, 'imdb_id']
    movies_df.loc[x, 'popularity'] = movies_df.loc[x, 'belongs_to_collection']
    movies_df.loc[x, 'belongs_to_collection'] = np.nan
    movies_df.loc[x, 'imdb_id'] = np.nan
    movies_df.loc[x, 'production_companies'] = movies_df.loc[x, 'genres']
    movies_df.loc[x, 'genres'] = np.nan

In [28]:
'''Aplicamos la función a las 3 filas mencionadas:'''

repair(19730)
repair(29503)
repair(35587)

In [29]:
'''Realizamos el cambio de tipo de la columna budget, id y popularity:'''

movies_df['budget'] = movies_df['budget'].astype(float)
movies_df['id'] = movies_df['id'].astype(float)
movies_df['popularity'] = movies_df['popularity'].astype(float)

In [30]:
'''Unimos las columnas cast y crew de credits_df con movies_df a partir del id de las películas'''

movies_df = movies_df.merge(credits_df, on = 'id')

In [31]:
''' Ahora comencemos a desanidar las columnas. Primero belongs_to_collection. En este caso se dejará solo
    el nombre de la colección a la que pertenezca la película, los demás valores serán removidos.'''

lista_index = movies_df[movies_df['belongs_to_collection'].notnull()].index

for i in range(0, movies_df.shape[0]):
    if i not in lista_index:        
        movies_df.loc[i, 'name_collection'] = np.nan
    else:
        a = list(eval(movies_df.loc[i, 'belongs_to_collection']).values())                
        movies_df.loc[i, 'name_collection'] = a[1]

In [32]:
'''Eliminamos la columna belongs_to_collection, ya que no nos será de más utilidad'''

movies_df.drop(columns = ['belongs_to_collection'], inplace = True)

In [33]:
''' Ahora trabajamos con las demás columnas anidadas. Se crearan nuevas columnas tantos valores estén
    anidados. Esto para poder usar los valores anidados en el modelado.
    Se creará una función que facilite hacer este trabajo.'''

''' Este es la función para cambiar los valores de la columna por listas con los valores importantes 
    en los diccionarios anidados.'''
def separar_anidados(column, df, indice_valor):
    l = df[df[column].notnull()].index
    for i in range(0, df.shape[0]):
        column_list = []
        if i in l:
            column_list = eval(df.loc[i, column])    
            lista_valores = []
            for j in range(0, len(column_list)):
                lista_valores.append(list(column_list[j].values())[indice_valor])
            df.loc[[i], column] = pd.Series([lista_valores], index=df.index[[i]])  

In [34]:
'''Aplicamos la función creada a genres y production_companies, las columnas production_countries y spoken_languages
    también están anidadas, pero no se utilizarán en las etapas posteriores del proyecto, por lo tanto no
    se transforman y se eliminarán más adelante'''

separar_anidados('genres', movies_df, 1)
separar_anidados('production_companies', movies_df, 0)

In [35]:
'''Trabajamos con los valores anidados de la columna cast, se tomarán solo los nombre de los actores'''

cast_final_list = []
for i in range(0, movies_df.shape[0]): 
    cast_list =[]
    for l in eval(movies_df.loc[i, 'cast']):
        cast_list.append(l['name'])
    cast_final_list.append(cast_list)
cast_serie = pd.Series(cast_final_list)
movies_df['cast'] = cast_serie

In [36]:
''' Ahora trabajamos con la columna crew. Extraeremos solo el nombre de los directores de las películas.
    Luego crearemos una nueva columna llamada directors y eliminaremos la columna crew.'''

directors = []
for i in range(0, movies_df.shape[0]):
    l = eval(movies_df.loc[i, 'crew'])
    director_list = []
    for x in l:
        if x['job'] == 'Director':
            director_list.append(x['name'])
    directors.append(director_list)
director_serie = pd.Series(directors)
movies_df['directors'] = director_serie
movies_df.drop(columns = ['crew'], inplace = True)

In [37]:
'''Se reemplazan los valores nulos de las columnas revenue y budget por 0:'''

movies_df['budget'].fillna(0, inplace = True)
movies_df['revenue'].fillna(0, inplace = True)

In [38]:
'''Se ven cuantos valores nulos están presentes en la columna release_date'''

movies_df[movies_df['release_date'].isnull()]['adult'].count()

87

In [39]:
''' Como se observa, solo hay 87 valores nulos de 45466 valores en total, por esto se eliminan los datos nulos
    de release_date, pero antes se crea una lista con los id de las películas que se van a eliminar.'''

lista_eliminados = movies_df[movies_df['release_date'].isnull()]['id'].to_list()

movies_df.dropna(subset = ['release_date'], axis = 'index', inplace = True)

In [40]:
# Se cambia el tipo de datos de la columna release_date y se crea la columna release_year

movies_df['release_date'] = pd.to_datetime(movies_df['release_date']) # Cambio de formato a formato date
movies_df['release_year'] = movies_df['release_date'].dt.year # Creación de columna release_year
movies_df['release_year'] = movies_df['release_year'].astype(int) # Cambio de tipo de la columna release_year a int

In [41]:
# Se resetean los índices para no tener índices faltantes y se pueda recorrer el dataframe sin problemas.

movies_df = movies_df.reset_index()

In [42]:
'''Se crea la columna return con el retorno de la inversión, dividiendo revenue / budget.
    Como la columna budget tiene valores que son 0 y la división entre 0 no está definida, se llenará
    la columna return con 0 en estos casos.'''

for i in range(0, movies_df.shape[0]):
    if movies_df.loc[i, 'budget'] != 0:
        movies_df.loc[i, 'return'] = movies_df.loc[i, 'revenue'] / movies_df.loc[i, 'budget']
    else:
        movies_df.loc[i, 'return'] = 0
movies_df['return'].fillna(0, inplace = True)

In [43]:
'''Se eliminan las columnas que no se utilizarán'''

movies_df.drop(columns = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'production_countries', 'tagline', 'spoken_languages'], inplace = True)

In [44]:
'''Se elimina la columna index y se cambia el tipo de datos de la columna vote_count y vote_average a float'''

movies_df.drop(columns = ['index'], inplace = True)
movies_df['vote_count'] = movies_df['vote_count'].astype(float)
movies_df['vote_average'] = movies_df['vote_average'].astype(float)

In [45]:
'''Evaluamos los duplicados en la columna id'''
list(set(movies_df[movies_df.duplicated('id')]['id'].to_list()))

[5511.0,
 18440.0,
 23305.0,
 109962.0,
 99080.0,
 125458.0,
 141971.0,
 187156.0,
 13209.0,
 9755.0,
 132641.0,
 77221.0,
 199591.0,
 4912.0,
 15028.0,
 12600.0,
 8767.0,
 14788.0,
 25541.0,
 97995.0,
 105045.0,
 24023.0,
 168538.0,
 152795.0,
 110428.0,
 24026.0,
 298721.0,
 142563.0,
 265189.0,
 84198.0,
 11752.0,
 159849.0,
 11115.0,
 119916.0,
 43629.0,
 10991.0,
 3057.0,
 69234.0,
 116723.0,
 123634.0,
 157301.0,
 22649.0,
 42495.0]

In [46]:
''' Se comparará la fecha de las filas duplicadas. Si hay diferencia de fecha se dejará la mayor, porque
    representa una actualización. Si las fechas son las mismas entonces se borrará uno de los registros.'''

peliculas_duplicadas = list(set(movies_df[movies_df.duplicated('id')]['id'].dropna().to_list()))

for i in peliculas_duplicadas:
    index_list = list(movies_df[movies_df['id'] == i].index)
    index_save = movies_df.loc[index_list[0], 'release_date'].date()
    drop_index = []
    for j in range(1, len(index_list)):
        if index_save < movies_df.loc[index_list[j], 'release_date'].date():
            index_save = movies_df.loc[index_list[j], 'release_date'].date()
        else:
            drop_index.append(index_list[j])
    movies_df.drop(index = drop_index, inplace = True)    

In [47]:
'''Eliminamos los 2 valores nulos de la columna id, reseteamos los índices y borramos la columna index'''

movies_df.dropna(subset = ['id'], axis = 'index', inplace = True)
movies_df = movies_df.reset_index()
movies_df.drop(columns = ['index'], inplace = True)

In [48]:
'''Quitaremos los espacios de las columnas genres, cast y crew'''

movies_df['genres'] = movies_df['genres'].apply(lambda x: [i.replace(' ', '') for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x: [i.replace(' ', '') for i in x])
movies_df['directors'] = movies_df['directors'].apply(lambda x: [i.replace(' ', '') for i in x])

In [49]:
''' Rellenamos los espacios nulos de overview con strings vacios y convertimos la columna en una lista. Además, 
    creamos una nueva columna que contendrá el título en formato de lista'''

movies_df['overview'] = movies_df['overview'].fillna(' ')
movies_df['overview'] = movies_df['overview'].apply(lambda x: x.split())
movies_df['title_list'] = movies_df['title'].apply(lambda x: x.split())

In [50]:
''' Creamos una nueva columna llamada union que será la combinación de las columnas: título_list, overview, genres,
    cast, director y production_companies quitándole los espacios en blanco, y la convertimos a string. 
    Esta columna nos ayudará con el modelado del sistema de recomendación'''

movies_df['union'] = movies_df['title_list'] + movies_df['overview'] + movies_df['genres'] + movies_df['cast'] + movies_df['directors'] + movies_df['production_companies'].apply(lambda x: [i.replace(' ', '') for i in x])
movies_df['union'] = movies_df['union'].apply(lambda x: ' '.join(x))

In [51]:
''' Ahora trabajamos sobre el dataset de credtis.
    Eliminamos de credits_df los id de las películas que fueron eliminados en movies_df.'''

deleted_index = []

for i in range(0, credits_df.shape[0]):
    if credits_df.loc[i, 'id'] in lista_eliminados:
        deleted_index.append(i)

credits_df.drop(index = deleted_index, inplace = True)

In [52]:
'''Reseteamos los índices y borramos la columna index que se forma'''

credits_df = credits_df.reset_index()
credits_df.drop(columns = ['index'], inplace = True)

In [53]:
'''Se crearan 2 Dataframe nuevos, uno para el cast y otro para el crew. Se tomarán los datos más importantes
de cada una de las columnas para los nuevos dataframes. Una vez creados, se dejará de tomar en cuenta el 
dataframe de credits.

Se crea el dataframe del cast'''

cast_list = ['name']

cast_l = []
for j, y in credits_df.iterrows():    
    lista_valores = eval(credits_df.loc[j, 'cast'])
    for k in range(0, len(lista_valores)):
        value_list = []
        for x in cast_list:
            value_list.append(lista_valores[k][x])
        value_list.append(credits_df.loc[j, 'id'])
        cast_l.append(value_list)

cast_array = np.array(cast_l)
cast_df = pd.DataFrame(cast_array, columns = ['name', 'movie_id'])

In [54]:
'''Se crea el dataframe de crew'''

crew_l = []
for j in range(0, credits_df.shape[0]):    
    lista_valores = eval(credits_df.loc[j, 'crew'])
    for k in range(0, len(lista_valores)):
        value_list = []
        if lista_valores[k]['job'] == 'Director':
            value_list.append(lista_valores[k]['name'])
            value_list.append(credits_df.loc[j, 'id'])    
            crew_l.append(value_list)

crew_array = np.array(crew_l)
director_df = pd.DataFrame(crew_array, columns = ['name', 'movie_id'])


In [55]:
'''Guardamos los dataset limpios como archivos csv para usarlos en el EDA, modelado y API'''

movies_df.to_csv('Movies_limpio.csv', index = False)
cast_df.to_csv('Cast.csv', index = False)
director_df.to_csv('Directores.csv', index = False)